In [31]:
import numpy as np
from sklearn.preprocessing import StandardScaler


In [32]:
# running the shuffle method two times and you notice the the shuffle result changes because the shuffle 
# method is called twice resulting in using different shuffles.
# if you run the code from default_rng(42) then you will always get the same result
r = np.random.default_rng(42)
a = np.arange(10)
r.shuffle(a)
a

array([5, 6, 0, 7, 3, 2, 4, 9, 1, 8])

In [33]:
# raw_csv_data = pd.read_csv('Audiobooks_data.csv')
raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter=',')
unscalsed_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

In [34]:
targets_all.shape

(14084,)

#### Balance the dataset

In [35]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []
rng = np.random.default_rng(42)
# rng.shuffle(targets_all)
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter >= num_one_targets:
            indices_to_remove.append(i)
# strange thing is that the implanaced data as it is can lead to a model with a very high accuracy 0.9
# balanced but not shuffled before balancing also leads to a good model 0.86
unscalsed_inputs_equal_priors = unscalsed_inputs_all #np.delete(unscalsed_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = targets_all #np.delete(targets_all, indices_to_remove, axis=0)

#### Standardize the inputs

In [36]:
scaler = StandardScaler()
scaler.fit(unscalsed_inputs_equal_priors)
scaled_inputs = scaler.transform(unscalsed_inputs_equal_priors)

#### Shuffle data for improved batching

In [37]:
# store the indices in a variable so that inputs and targets are shuffled exactly the same
shuffled_indices = np.arange(scaled_inputs.shape[0])
rng.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

#### Split dataset into train, validation, and test
* try the model with a test data that is not balanced!

In [38]:
sample_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8*sample_count)
validation_samples_count = int(0.1*sample_count)
test_samples_count = sample_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:validation_samples_count+train_samples_count]
validation_targets = shuffled_targets[train_samples_count:validation_samples_count+train_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1788.0 11267 0.15869352977722553
225.0 1408 0.15980113636363635
224.0 1409 0.1589779985805536


#### Save the three datasets in *.npz

In [39]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

In [40]:
train_inputs.shape

(11267, 10)